### Importing Libraries

In [1]:
import gurobipy as gp
from gurobipy import *
import numpy as np
import pandas as pd
from datetime import datetime

### Reading Data

In [2]:
# Reading Data
distances_data = pd.read_csv(r"D:\Uzair's Files\OneDrive - McGill University\Fall Semester\MGSC 662 - Decision Analytics\Assignments\Project\Stops_distances_Final_Subset.csv", index_col = 0)
distances_data.head()

,53696,51278,60719,60720,60726,60728,60868,60717,50102,50103,...,61906,61907,61908,61909,61923,61990,61991,61992,61993,62042
Stops,,,,,,,,,,,,,,,,,,,,,
53696,0.000000,4723.734076,1810.955716,2499.736931,1597.766400,1434.429762,2605.614922,1378.287636,6440.288601,6582.647862,...,6117.277841,6041.858711,6344.565827,6680.461026,3458.768954,5684.567936,5757.051564,4712.792971,4197.443117,7863.893575
51278,4784.049431,0.000000,6595.005146,7267.582061,6381.815830,6218.479192,7373.460052,6162.337066,6008.070385,6150.429645,...,5579.829424,5504.410294,5628.466223,5596.451079,8138.211141,3493.987042,3288.097146,4530.388416,4656.825678,7059.928265
60719,1814.716319,6538.450395,0.000000,285.293679,219.952406,487.467750,419.117733,571.469028,8840.376607,8982.735868,...,5287.182320,5211.763190,5514.470305,5850.365505,2781.080594,5835.311773,5907.795401,7112.880976,6597.531123,10263.981580
60720,2078.087649,6801.821725,285.293679,0.000000,505.246085,772.761429,133.824053,856.762707,8987.196728,9129.555988,...,5041.051772,4965.632642,5268.339757,5604.234957,2927.900715,6098.683103,6171.166731,7259.701097,6744.351244,10410.801700
60726,1603.202900,6326.936977,219.952406,505.246085,0.000000,283.610681,639.070139,367.611959,8628.863189,8771.222449,...,5238.951200,5163.532070,5466.239185,5802.134385,2569.567175,5623.798354,5696.281982,6901.367558,6386.017705,10052.468160


In [3]:
dist_array = np.asarray(distances_data.iloc[:200, :200]) # Filtering for the relevant 200 stops
print("Shape of the stop-distances matrix is:", dist_array.shape)

Shape of the stop-distances matrix is: (200, 200)


### Adjusting Parameters

In [4]:
# Adjustable Parameters
n = len(dist_array) # Number of Stops
max_stops_per_bus = 79
min_stops_per_bus = 4
min_buses = 5
max_buses = 15

# Non-Adjustable Parameters
L = min(max_stops_per_bus, n) # Maximum Number of Stops Visited by a single bus
min_stop = min(min_stops_per_bus, n) # Minimum allowable stops for a bus
K = min(min_stop, L) # Minimum Number of Stops Visited by a single bus
m_min = max(3, int(n/L + 1), min_buses) # Minimum number of buses we can have
m_max = max(int(n/L + 1), max_buses) # Maximum number of buses we can have
print(f"Number of Stops (n): {n}\nMaximum number of stops visited by a single bus (L): {L} \nMinimum number of stops a bus can visit (K): {K} \nMaximum number of buses we can operate (m_max): {m_max} \nMinimum number of buses we can operate (m_min): {m_min} ")

Number of Stops (n): 200
Maximum number of stops visited by a single bus (L): 79 
Minimum number of stops a bus can visit (K): 4 
Maximum number of buses we can operate (m_max): 15 
Minimum number of buses we can operate (m_min): 5 


### Checking for error in model definition

In [5]:
# Error Logs
if n < 5:
    raise ValueError('Too few stops. Something is wrong with the distance array')
elif min_buses < n/max_stops_per_bus:
    raise ValueError(f'ERROR: Increase the minimum buses we can have to at least {int(n/max_stops_per_bus)+1}')
elif L<K:
    raise ValueError('ERROR: Change the minimum and maximum stops')
elif max_buses >= int((n-1)/K):
    raise ValueError(f'ERROR: Value too high for max_buses, threshold value {int((n-1)/K)-1}. min_stops_per_bus could also be too high.')
elif min_stop >= L:
    raise ValueError(f'ERROR: Minimum stops for a single bus exceeds total stops')
elif m_max < m_min:
    raise ValueError(f'ERROR: Maximum buses are less than minimum buses. Change min_buses to < {m_max}')
elif n<3:
    raise ValueError('ERROR: Stops should be greater than 2')
elif L < K:
    raise ValueError('ERROR: Maximum number of possible stops should be greater than minimum number of stops')
elif 2 > K or K > ((n-1)/m_max):
    raise ValueError('ERROR: Re-adjust values to satisfy 2 <= K <= (n-1)/m_max')
elif L < K:
    raise ValueError('ERROR: Maximum number of possible stops should be greater than minimum number of stops')
elif n/L > m_max:
    raise ValueError(f'ERROR: Increase the maximum allowed number of buses at least {int(n/L + 1)}')
elif m_min < 0:
    raise ValueError(f'Need to Increase the maximum allowed number of buses or change minimum buses threshold, current={search_threshold}')
else:
    print("C'est Bon!")

C'est Bon!


In [6]:
# Finding which Old Port Stops we want one of our buses to visit in a single route
oldport_stops =  [distances_data.index.get_loc(col) for col in [60868, 60720, 60719, 60726, 60728, 60717]]

# From St Antoine/ de la Cathédrale as it is the bus stop the nearest to the train terminal at Decelles/ Queen Mary 
trainterminal_stops =  [distances_data.index.get_loc(col) for col in [53696, 51278]]

### Initializing Model

In [8]:
model = gp.Model("Bus_Route_Optimization")

######################
## Defining Decision Variables
######################
x = model.addVars(n, n, vtype = GRB.BINARY, name = [str(i)+">"+str(j) for i in range(n) for j in range(n)])
u = model.addVars(n, vtype = GRB.INTEGER, lb = 1, ub=n-1, name = "u")

######################
## Cost Calculation
######################
"""
# Fuel cost = 1.464 Canadian $ per liter 
# Mileage = 47.5 liters / 100 kms 
# Cost to travel 1 meter (cost_m) = 47.5/(100x1000) x 1.464 = 0.0006954 $/meter
# Cost of procuring each bus (cost_bus) = 1.13 million dollars
# Maintenance cost per meter (maintanance_m) = 0.003 $/meter
# For traversing the network for 6 months (with every route traversed on average of 15 mins, operating 5am till 12am)
# cost_travel = cost_travel_route * hours_operation_day * routes_hour * days
"""
cost_m, hours_operation_day, bus_freq, days, bus_cost  = 0.0006954, 19, 15, 180, 1130000 
cost_travel = gp.quicksum(dist_array[i,j]*x[i,j]*cost_m for i in range(n) for j in range(n)) * hours_operation_day * (60/bus_freq) * (days)
cost_buses = gp.quicksum(x[0,j] * bus_cost for j in range(n))

######################
## Objective Function
######################
model.setObjective(cost_travel + cost_buses, GRB.MINIMIZE) # We want to minimize the total cost

######################
## Constraints
######################
# All buses start from terminus stop [0]
model.addConstr(gp.quicksum(x[0, j] for j in range(1, n)) >= m_min, name = 'Minimum buses start from terminus node')
model.addConstr(gp.quicksum(x[0, j] for j in range(1, n)) <= m_max, name = 'Maximum buses start from terminus node')

# All buses end at terminus stop [0]
model.addConstr(gp.quicksum(x[j, 0] for j in range(1, n)) >= m_min, name = 'Minimum buses end at terminus node') # ensures at least 2 buses enter the terminus
model.addConstr(gp.quicksum(x[j, 0] for j in range(1, n)) <= m_max, name = 'Maximum buses end at terminus node') # ensures at most m buses enter the terminus

# Each stop exited by only one bus
for i in range(1, n):
    model.addConstr(gp.quicksum(x[i,j] for j in range(n)) == 1, name = 'Each Stop Exited by only 1 bus')
    
# Each stop entered by only one bus
for j in range(1, n):
    model.addConstr(gp.quicksum(x[i,j] for i in range(n)) ==1, name = 'Each Stop Entered by only 1 bus')

# To not get stuck at the same stop
for i in range(n):
    model.addConstr(x[i,i]==0)

# Sub-tour elimination: Lifted Kulkarni–Bhave SECs
for i in range(1,n):
    for j in range(1,n):
        if i==j:
            continue
        else:
            model.addConstr(u[i]-u[j]+ L*x[i,j] + (L-2)*x[j,i] <= L-1, name = 'Sub-tour elimination')
            
# Ensure that buses visit at most L stops and at least K stops 
for i in range(1,n):
    model.addConstr(u[i] + (L-2) * x[0, i] - x[i, 0] <= L-1, name = 'Visit at most L stops')
    model.addConstr(u[i] + x[0, i] + (2-K) * x[i, 0] >= 2, name = 'Visit at least K stops')
    if K < 4:
        model.addConstr(x[0,i] + x[i,0] <= 1) # Constraint is redundant for K >= 4

# One of the buses should visit the stops along the old port
for i in range(len(oldport_stops)-1):
    model.addConstr(x[oldport_stops[i], oldport_stops[i+1]] == 1, name = 'Old Port Route')
    
# One of the buses should visit the stops from St Antoine/ de la Cathédrale to the train terminal at Decelles/ Queen Mary
for i in range(len(trainterminal_stops)-1):
    model.addConstr(x[trainterminal_stops[i], trainterminal_stops[i+1]] == 1, name = 'Train Terminal wo St Antoine/ de la Cathédrale')

### Running the optimization

In [9]:
######################
## Running the optimization
######################

model.Params.MIPGap = 0.01 # Ranges from 0 to 1. At 0.01 or 1% gap we want to stop the solver.
model.Params.MIPFocus = 3 # To get a faster solution
model.Params.Cuts = 3 # Cutting Planes ensure a faster way to solve complex problems
model.Params.Presolve = 2 # Presolving reduces the size of the problem by substitution and eliminating redundant constraints 

time_1 = datetime.now() # Start-time
model.optimize() # Run model optimization
time_2 = datetime.now() # End-time

Changed value of parameter MIPGap to 0.01
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter MIPFocus to 3
   Prev: 0  Min: 0  Max: 3  Default: 0
Changed value of parameter Cuts to 3
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter Presolve to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 40408 rows, 40200 columns and 239404 nonzeros
Model fingerprint: 0x4709e55b
Variable types: 0 continuous, 40200 integer (40000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [3e+01, 1e+06]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 8e+01]
Presolve removed 1805 rows and 2372 columns (presolve time = 6s) ...
Presolve removed 1805 rows and 2372 columns
Presolve time: 9.60s
Presolved: 38603 rows, 37828 columns, 338513 nonzeros
Variable types: 0 continuou

### Results

In [147]:
# Calculating Time Difference
delta_t = time_2 - time_1
print("Time Elapsed (Hours:Minutes:Seconds): ", str(delta_t))

Time Elapsed (Hours:Minutes:Seconds):  0:38:27.344269


In [154]:
# Code to get the stops traversed and their lengths
starting_paths = []
for var in model.getVars():
    if  var.varName.startswith('0>') and var.X > 0:
        starting_paths.append(var.varName)
d_starting_paths = {}
for i in starting_paths:
    d_starting_paths[i] = None

def cycle(filter_list):
    cycle = ['53696', ' > ']
    k = 0
    for i in range(1,n):
        for j in range(1,n):
            name = str(k)+'>'+str(j)
            if name not in filter_list:
                if model.getVarByName(name).x >0:
                    cycle.append(distances_data.columns[j])
                    cycle.append(" > ")
                    k=j
    cycle.append(cycle[0])
    return(cycle)
keys_list = list(d_starting_paths.keys())
k=1
print('\x1b[1;31m'+'Routes Traversed:'+'\x1b[0m')
full_distance = 0
for i in keys_list:
    filter_list = [j for j in keys_list if i!=j]
    d_starting_paths[i] = cycle(filter_list)
    stops = [i for i in d_starting_paths[i] if i != ' > ']
    distance = 0
    for j in range(len(stops)-1):
        distance += dist_array[distances_data.columns.get_loc(stops[j]), distances_data.columns.get_loc(stops[j+1])]
    print("\nBus ", k," (", len(stops)-2,  f" Stops, {distance/1000:.2f} km):\n", *d_starting_paths[i], sep="")
    full_distance += distance
    k+=1

Routes Traversed:

Bus 1 (35 Stops, 19.27 km):
53696 > 51278 > 50789 > 50810 > 50807 > 50753 > 50721 > 50690 > 50654 > 50580 > 50582 > 50587 > 50583 > 50655 > 50691 > 50683 > 50663 > 50680 > 50629 > 50630 > 50631 > 50701 > 50722 > 50745 > 50744 > 50778 > 50777 > 50697 > 50738 > 50737 > 50736 > 50818 > 50817 > 50819 > 50820 > 50791 > 53696

Bus 2 (6 Stops, 5.00 km):
53696 > 60868 > 60720 > 60719 > 60726 > 60728 > 60717 > 53696

Bus 3 (4 Stops, 7.80 km):
53696 > 50111 > 50112 > 50115 > 50123 > 53696

Bus 4 (79 Stops, 47.85 km):
53696 > 50114 > 50116 > 50104 > 50821 > 50822 > 50735 > 50746 > 50692 > 50694 > 50693 > 50608 > 50609 > 50581 > 50547 > 50548 > 50534 > 50533 > 50560 > 50476 > 50500 > 50532 > 50509 > 50531 > 50508 > 50599 > 50607 > 50598 > 50550 > 50544 > 50584 > 50642 > 50641 > 50643 > 50714 > 50795 > 50728 > 50658 > 50660 > 50659 > 50703 > 50684 > 50702 > 50806 > 50790 > 50700 > 50628 > 50591 > 50510 > 50475 > 50435 > 50477 > 50478 > 50479 > 50397 > 50398 > 50436 > 50410 > 5037

In [1]:
print('\x1b[1;31m'+'Key Results:'+'\x1b[0m')
print(f"Total length of all routes: {full_distance/1000:.2f} km \nObjective Value: ${int(model.objVal):,} \nCost of purchasing {k-1} buses: ${(k-1)*bus_cost:,} \nCost of traversing the route for 6 months: ${int(model.objVal-(k-1)*bus_cost):,}")
print('\x1b[1;31m'+'\nCustom Route:'+'\x1b[0m')
print("- Bus#1 starts from St Antoine/ de la Cathédrale to the train terminal at Decelles/ Queen Mary")
print("- Bus#2 travels along the old port")

Key Results:
Total length of all routes: 124.67 km 
Objective Value: $7,080,716 
Cost of purchasing 5 buses: $$5,650,000  
Cost of traversing the route for 6 months: $$1,430,716

Custom Route:
- Bus#1 starts from St Antoine/ de la Cathédrale to the train terminal at Decelles/ Queen Mary
- Bus#2 travels along the old port


### Exporting Data

In [141]:
df = pd.DataFrame([[var.varName, var.X] for var in model.getVars()], columns = ['Variable', 'Value'])
df.to_csv('./bus_variables_fewstops.csv', encoding='utf-8', index=False)

In [142]:
model.write('bus_model_fewstops.mps')